# Create as webpage

<img src="Create_Webpage.webp" style="width:280px; height:280px;">

This page describes and actually impliments, how to make the jupyter notebooks of the whole project a working web page. Please note the prerequisite of having generated all needed html files, by executing the according scripts in [Create PDF](Create_pdf.en.ipynb).

## Copy all resources

In this step all html files and all needed assets are copied to a temporary folder.

In [1]:
import os
import shutil
from bs4 import BeautifulSoup

# Exclusion lists
EXCLUDE_FILES = ["Everybody_Can_Code.html"]  # Add filenames to exclude
EXCLUDE_DIRS = ["website", "drafts"]            # Add directory names to exclude

# Function to ensure a directory exists
def ensure_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Function to create an index.html with a redirect
def create_redirect_index_html(target_file, dst_dir):
    index_path = os.path.join(dst_dir, "index.html")

    redirect_content = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta http-equiv="refresh" content="0; url={target_file}">
        <title>Redirecting...</title>
    </head>
    <body>
        <p>Redirecting to <a href="{target_file}">{target_file}</a>...</p>
    </body>
    </html>
    """

    with open(index_path, "w", encoding="utf-8") as index_file:
        index_file.write(redirect_content)

# Function to copy all HTML files and their resources, excluding specific files and directories
def copy_all_html_and_resources(src_dir, dst_dir):
    ensure_dir(dst_dir)

    for root, dirs, files in os.walk(src_dir):
        # Exclude directories
        dirs[:] = [d for d in dirs if d not in EXCLUDE_DIRS]

        for file in files:
            # Skip excluded files
            if file in EXCLUDE_FILES:
                continue

            if file.endswith(".html"):
                src_html_path = os.path.join(root, file)

                # Copy HTML file
                relative_path = os.path.relpath(root, src_dir)
                target_dir = os.path.join(dst_dir, relative_path)
                ensure_dir(target_dir)

                dst_html_path = os.path.join(target_dir, file)
                shutil.copy2(src_html_path, dst_html_path)

                # Analyze and copy resources
                with open(src_html_path, "r", encoding="utf-8") as html_file:
                    soup = BeautifulSoup(html_file, "html.parser")
                    for img in soup.find_all("img"):
                        if img.get("src"):
                            original_path = os.path.normpath(os.path.join(root, img["src"]))
                            if os.path.isfile(original_path):
                                resource_target_dir = os.path.join(target_dir, os.path.dirname(img["src"]))
                                ensure_dir(resource_target_dir)
                                shutil.copy2(original_path, os.path.join(resource_target_dir, os.path.basename(img["src"])))

if __name__ == "__main__":
    # Variables for source directory, destination directory, and HTML file
    source_directory = "."
    destination_directory = "./website"
    source_html_file = "./programming/abstract/Intro.de.html"

    source_html_path = os.path.join(source_directory, source_html_file)

    if not os.path.exists(source_directory):
        print("Das Quellverzeichnis existiert nicht.")
    elif not os.path.isfile(source_html_path):
        print("Die angegebene Quelldatei existiert nicht.")
    else:
        shutil.rmtree(destination_directory)
        copy_all_html_and_resources(source_directory, destination_directory)
        create_redirect_index_html(source_html_file, destination_directory)
        print("Kopieren und Redirect abgeschlossen.")


Kopieren und Redirect abgeschlossen.


## Create menu and search

In this step a menu is created from the titles of the html pages and the folder structure they are in.

In [2]:
import os
import json
from bs4 import BeautifulSoup

# Configuration
BASE_DIR = "./website"  # Base directory containing the HTML files
SEARCH_INDEX_FILE = os.path.join(BASE_DIR, "search-index.json")
STYLE_FILE = os.path.join(BASE_DIR, "style.css")
URL_PREFIX = "/SoProMing/JederKannCoden/"  # Set the static prefix for URLs (e.g., "/subdir/")


def normalize_url(url):
    """Normalize URLs by removing ../ and resolving redundant parts."""
    # Remove "../" and normalize the path
    clean_url = os.path.normpath(url).replace("\\", "/")
    clean_url = clean_url.replace("../", "")
    # Ensure the URL starts with the configured prefix
    if not clean_url.startswith(URL_PREFIX):
        clean_url = f"{URL_PREFIX}{clean_url.lstrip('/')}"
    return clean_url


# Exclusion lists
EXCLUDE_FILES = ["index.html", "exclude.html"]  # Add filenames you want to exclude
EXCLUDE_DIRS = ["website"]                      # Add directory names you want to exclude


def sanitize_title(title):
    """Remove unwanted symbols like ¶ from the title."""
    if title:
        return title.replace("¶", "").strip()
    return title


def get_title_from_h1(file_path):
    """Extract the text of the first <h1> tag in the HTML file and sanitize it."""
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            soup = BeautifulSoup(file, "html.parser")
            h1 = soup.find("h1")
            return sanitize_title(h1.get_text(strip=True)) if h1 else None
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None


def extract_readable_text(file_path):
    """Extract readable text from an HTML file, ignoring non-visible elements."""
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            soup = BeautifulSoup(file, "html.parser")
            # Remove script and style elements
            for script_or_style in soup(["script", "style"]):
                script_or_style.decompose()

            # Extract visible text
            return soup.get_text(separator=" ", strip=True)
    except Exception as e:
        print(f"Error extracting text from {file_path}: {e}")
        return ""


def build_menu_tree(base_dir):
    """Build a nested dictionary representing the menu structure."""
    menu_tree = {}

    for root, dirs, files in os.walk(base_dir):
        # Remove excluded directories from traversal
        dirs[:] = [d for d in dirs if d not in EXCLUDE_DIRS]

        rel_dir = os.path.relpath(root, base_dir)
        current_level = menu_tree

        # Navigate to the current directory level in the tree
        if rel_dir != ".":
            for part in rel_dir.split(os.sep):
                current_level = current_level.setdefault(part, {})

        # Add files to the current level
        for file in sorted(files):
            if file.endswith(".html") and file not in EXCLUDE_FILES:
                file_path = os.path.join(root, file)
                title = get_title_from_h1(file_path)
                if title:  # Only include files with a valid <h1> title
                    current_level[file] = title

    return menu_tree

def generate_menu_html(menu_tree, current_path="", relative_to="", level=0):
    """Generate HTML for the menu structure recursively with nested <ul><li>,
    only including folders on the top level, and skipping empty folders."""
    def is_non_empty_folder(value):
        return isinstance(value, dict) and any(is_non_empty_folder(v) or isinstance(v, str) for v in value.values())

    html = '<ul class="{}">\n'.format("menu" if level == 0 else "submenu")
    for key, value in sorted(menu_tree.items()):
        if level == 0 and not isinstance(value, dict):
            continue  # Skip files on top level

        if isinstance(value, str):  # File
            rel_path = os.path.join(current_path, key)
            relative_link = normalize_url(os.path.relpath(rel_path, relative_to).replace("\\", "/"))
            html += f'  <li><a href="{relative_link}">{value}</a></li>\n'
        elif is_non_empty_folder(value):  # Non-empty folder
            title = key.capitalize()
            html += f'  <li class="menu-item">\n'
            html += f'    <a href="#">{title}</a>\n'
            html += generate_menu_html(value, os.path.join(current_path, key), relative_to, level + 1)
            html += '  </li>\n'
    html += '</ul>\n'
    return html

def generate_search_index(base_dir):
    """Generate a JSON file for the search index."""
    search_index = []

    for root, dirs, files in os.walk(base_dir):
        # Remove excluded directories from traversal
        dirs[:] = [d for d in dirs if d not in EXCLUDE_DIRS]

        for file in files:
            if file.endswith(".html") and file not in EXCLUDE_FILES:
                absolute_path = os.path.relpath(os.path.join(root, file), base_dir).replace("\\", "/")
                file_path = os.path.join(root, file)
                title = get_title_from_h1(file_path)
                if title:  # Only include files with a valid <h1> title
                    readable_text = extract_readable_text(file_path)
                    search_index.append({
                        "url": normalize_url(f"{absolute_path}"),  # Normalize the URL
                        "title": title,
                        "content": readable_text  # Use readable text only
                    })

    # Save the search index
    with open(SEARCH_INDEX_FILE, "w", encoding="utf-8") as f:
        json.dump(search_index, f, indent=4, ensure_ascii=False)
    print(f"Search index created: {SEARCH_INDEX_FILE}")


def generate_style():
    """Generate a CSS file for styling."""
    style_content = """
    .menu {
    display: flex;
    list-style: none;
    padding: 0;
    margin: 0;
    background-color: #f8f8f8;
    gap: 20px;
    }

    .menu-item {
    position: relative;
    }

    .menu > .menu-item > a {
    display: block;
    padding: 10px 16px;
    text-decoration: none;
    color: black;
    }

    .submenu {
    display: none;
    position: absolute;
    top: 100%;
    left: 0;
    list-style: none;
    background-color: white;
    padding: 0;
    margin: 0;
    min-width: 160px;
    box-shadow: 0 2px 8px rgba(0,0,0,0.15);
    z-index: 1000;
    }

    .submenu li a {
    display: block;
    padding: 10px 16px;
    text-decoration: none;
    color: black;
    white-space: nowrap;
    }

    .submenu li a:hover {
    background-color: #f0f0f0;
    }

    /* Show submenu on hover */
    .menu-item:hover > .submenu {
    display: block;
    }

    /* Nested submenu (submenu of submenu) */
    .submenu .submenu {
    top: 0;
    left: 100%;
    }

    .menu,
    .menu li,
    .menu a,
    .submenu li,
    .submenu a {
        font-family: Arial, sans-serif;
    }

    #search-container {
        position: relative;
        display: flex;
        flex-direction: column;
        align-items: flex-end;
    }

    #searchResults {
        display: none;
        position: absolute;
        top: 100%;
        right: 0;
        background: white;
        box-shadow: 0 2px 8px rgba(0,0,0,0.15);
        margin-top: 5px;
        width: 200px;
        max-height: 300px;
        overflow-y: auto;
        z-index: 1000;
    }

    #searchResults.show {
        display: block;
    }

    #searchResults div {
        padding: 4px 8px;
        cursor: pointer;
    }

    #searchResults div:hover {
        background-color: #f0f0f0;
    }
    """
    with open(STYLE_FILE, "w", encoding="utf-8") as f:
        f.write(style_content)
    print(f"CSS created: {STYLE_FILE}")


def inject_menu_and_search(base_dir, menu_tree):
    """Inject the menu and search functionality into each HTML file."""
    for root, dirs, files in os.walk(base_dir):
        # Remove excluded directories from traversal
        dirs[:] = [d for d in dirs if d not in EXCLUDE_DIRS]

        for file in files:
            if file.endswith(".html") and file not in EXCLUDE_FILES:
                file_path = os.path.join(root, file)
                relative_to = root

                # Generate menu HTML specific to the current file
                menu_html = f"""
                {generate_menu_html(menu_tree, current_path="", relative_to=relative_to)}
                """

                # Adjust paths for style and search index
                relative_path_to_base = os.path.relpath(BASE_DIR, root)
                style_path = normalize_url(os.path.join(relative_path_to_base, "style.css").replace("\\", "/"))
                search_index_path = normalize_url(os.path.join(relative_path_to_base, "search-index.json").replace("\\", "/"))

                with open(file_path, "r", encoding="utf-8") as f:
                    content = f.read()

                # Add the menu and search functionality to the HTML file
                new_content = f"""
                <!DOCTYPE html>
                <html lang="en">
                <head>
                    <meta charset="UTF-8">
                    <meta name="viewport" content="width=device-width, initial-scale=1.0">
                    <link rel="stylesheet" href="{style_path}">
                    <title>{get_title_from_h1(file_path) or "Untitled"}</title>
                </head>
                <body>
                    <nav>
                        <div class="menu">
                            {menu_html}
                        </div>
                        <div id="search-container">
                            <input type="text" id="searchInput" placeholder="Search...">
                            <div id="searchResults"></div>
                        </div>
                    </nav>
                    <div class="content">
                        {content}
                    </div>
                    <script>
                        const input = document.getElementById("searchInput");
                        const resultsContainer = document.getElementById("searchResults");

                        input.addEventListener("input", async function () {{
                            try {{
                                const query = this.value.toLowerCase();
                                if (!query) {{
                                    resultsContainer.innerHTML = "";
                                    resultsContainer.classList.remove("show");
                                    return;
                                }}

                                const response = await fetch("{search_index_path}");
                                const pages = await response.json();

                                const results = pages.filter(page =>
                                    page.title.toLowerCase().includes(query) || page.content.toLowerCase().includes(query)
                                );

                                resultsContainer.innerHTML = "";
                                results.forEach(result => {{
                                    const resultElement = document.createElement("div");
                                    resultElement.innerHTML = `<a href="${{result.url}}">${{result.title}}</a>`;
                                    resultsContainer.appendChild(resultElement);
                                }});

                                resultsContainer.classList.toggle("show", results.length > 0);

                            }} catch (error) {{
                                console.error("Search error:", error);
                            }}
                        }});

                        // Optional: hide results when clicking outside
                        document.addEventListener("click", (event) => {{
                            if (!document.getElementById("search-container").contains(event.target)) {{
                                resultsContainer.classList.remove("show");
                            }}
                        }});
                    </script>
                </body>
                </html>
                """

                with open(file_path, "w", encoding="utf-8") as f:
                    f.write(new_content)
                print(f"Menu and search added: {file_path}")

if __name__ == "__main__":
    if not os.path.exists(BASE_DIR):
        print(f"The directory {BASE_DIR} does not exist. Please create or adjust it.")
    else:
        generate_style()
        menu_tree = build_menu_tree(BASE_DIR)
        generate_search_index(BASE_DIR)
        inject_menu_and_search(BASE_DIR, menu_tree)
        # Copy JederKannCoden.pdf to the website directory
        pdf_source = "../JederKannCoden.pdf"
        pdf_destination = os.path.join(BASE_DIR, "JederKannCoden.pdf")
        if os.path.exists(pdf_source):
            shutil.copy2(pdf_source, pdf_destination)
            print(f"PDF copied to {pdf_destination}")
        else:
            print(f"PDF source file {pdf_source} does not exist.")


CSS created: ./website/style.css
Search index created: ./website/search-index.json
Menu and search added: ./website/Deploy_Contents.en.html
Menu and search added: ./website/Create_pdf.en.html
Menu and search added: ./website/Create_Html_Tree.html
Menu and search added: ./website/htw/Team_Signup.de.html
Menu and search added: ./website/tools/GIT.de.html
Menu and search added: ./website/tools/UTM_n_VS.de.html
Menu and search added: ./website/programming/Number_Systems.de.html
Menu and search added: ./website/programming/Programmieren1.de.html
Menu and search added: ./website/programming/Objectoriented_Programming.de.html
Menu and search added: ./website/programming/Systemlevel_Programming_Wintel.de.html
Menu and search added: ./website/programming/WebAPIs.de.html
Menu and search added: ./website/programming/Programming_And_TicTacToe.de.html
Menu and search added: ./website/programming/abstract/Intro.de.html
Menu and search added: ./website/programming/abstract/Contents.de.html
Menu and s

## Upload to website

In this final step all data is transfered according to settings in a `.env` file.

In [ ]:
%pip install paramiko

In [3]:
import os
import paramiko
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Configuration
SFTP_HOST = os.getenv("SFTP_HOST")
SFTP_PORT = int(os.getenv("SFTP_PORT", 22))  # Default port 22
SFTP_USERNAME = os.getenv("SFTP_USERNAME")
SFTP_PASSWORD = os.getenv("SFTP_PASSWORD")
REMOTE_DIR = os.getenv("REMOTE_DIR")
LOCAL_DIR = os.getenv("LOCAL_DIR")

def ensure_remote_dir(sftp, remote_path):
    """
    Ensure that the remote directory exists, creating it if necessary.
    """
    directories = remote_path.strip("/").split("/")
    current_dir = ""
    for directory in directories:
        current_dir += f"/{directory}"
        try:
            sftp.stat(current_dir)
        except FileNotFoundError:
            sftp.mkdir(current_dir)


def upload_directory(sftp, local_dir, remote_dir):
    """
    Upload the contents of a local directory to a remote SFTP directory.
    """
    for root, dirs, files in os.walk(local_dir):
        # Calculate the relative path and corresponding remote path
        relative_path = os.path.relpath(root, local_dir)
        remote_path = os.path.join(remote_dir, relative_path).replace("\\", "/")

        # Ensure the remote directory exists
        ensure_remote_dir(sftp, remote_path)

        # Upload files
        for file in files:
            local_file = os.path.join(root, file)
            remote_file = os.path.join(remote_path, file).replace("\\", "/")
            print(f"Uploading {local_file} to {remote_file}...")
            sftp.put(local_file, remote_file)

def main():
    """
    Main function to handle the SFTP upload process.
    """
    if not all([SFTP_HOST, SFTP_USERNAME, SFTP_PASSWORD, REMOTE_DIR, LOCAL_DIR]):
        print("Error: Missing required environment variables in .env file.")
        return

    # Establish SFTP connection
    try:
        print("Connecting to SFTP server...")
        transport = paramiko.Transport((SFTP_HOST, SFTP_PORT))
        transport.connect(username=SFTP_USERNAME, password=SFTP_PASSWORD)

        sftp = paramiko.SFTPClient.from_transport(transport)
        print("Connection established.")

        # Upload the directory
        print(f"Uploading contents of {LOCAL_DIR} to {REMOTE_DIR}...")
        upload_directory(sftp, LOCAL_DIR, REMOTE_DIR)
        print("Upload complete.")

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        if 'sftp' in locals():
            sftp.close()
        if 'transport' in locals():
            transport.close()
        print("Connection closed.")

if __name__ == "__main__":
    main()

Connecting to SFTP server...
Connection established.
Uploading contents of ./website to /customers/9/d/3/2hands.de/httpd.www/www8/SoProMing/JederKannCoden...
Uploading ./website/index.html to /customers/9/d/3/2hands.de/httpd.www/www8/SoProMing/JederKannCoden/./index.html...
Uploading ./website/Create_Webpage.webp to /customers/9/d/3/2hands.de/httpd.www/www8/SoProMing/JederKannCoden/./Create_Webpage.webp...
Uploading ./website/search-index.json to /customers/9/d/3/2hands.de/httpd.www/www8/SoProMing/JederKannCoden/./search-index.json...
Uploading ./website/JederKannCoden.pdf to /customers/9/d/3/2hands.de/httpd.www/www8/SoProMing/JederKannCoden/./JederKannCoden.pdf...
Uploading ./website/Deploy_Contents.en.html to /customers/9/d/3/2hands.de/httpd.www/www8/SoProMing/JederKannCoden/./Deploy_Contents.en.html...
Uploading ./website/Create_pdf.en.html to /customers/9/d/3/2hands.de/httpd.www/www8/SoProMing/JederKannCoden/./Create_pdf.en.html...
Uploading ./website/style.css to /customers/9/d/3/